In [1]:
from pydbgen import pydbgen
import os
import pandas as pd
from faker import Faker
fake = Faker()
import random
import numpy as np
import string
import os
from tqdm import tqdm
import getpass

In [8]:
uname = 'postgres'
passwd = getpass.getpass()

········


# Postgres Test

In [3]:
import psycopg2

In [4]:
def table_exists(con, table_str):

    exists = False
    try:
        cur = con.cursor()
        cur.execute("select exists(select relname from pg_class where relname='" + table_str + "')")
        exists = cur.fetchone()[0]
        print (exists)
        cur.close()
    except psycopg2.Error as e:
        print (e)
    return exists

def get_table_col_names(con, table_str):

    col_names = []
    try:
        cur = con.cursor()
        cur.execute("select * from " + table_str + " LIMIT 0")
        for desc in cur.description:
            col_names.append(desc[0])        
        cur.close()
    except psycopg2.Error as e:
        print (e)

    return col_names

In [9]:
conn = psycopg2.connect(f"host='127.0.0.1' dbname='synthetic_email' user='{uname}' password='{passwd}'")

In [10]:
cur = conn.cursor()

In [11]:
cur.execute('SELECT count(*) FROM employees')

In [12]:
size = cur.fetchone()

In [13]:
size[0]

48801

In [14]:
!head /encrypted_vol/jupyter_data/airflow/data/employees.csv

"name","email","phone_number","address"
"Chad Spears","zacharyerickson@yahoo.com","557-303-4631","742 Jessica Cliffs, Port Stephenstad, TN 06314"
"Michael King","pmorales@yahoo.com","421-414-4215","909 Elizabeth Spring Apt. 692, Hessbury, MO 03015"
"Sharon Perez","michael58@harper.com","492-754-0699","6334 Martin Underpass, Lake Jeffrey, DC 59056"
"Susan Martinez","casey68@hotmail.com","741-792-0698","86052 Gregory Key, Lake Jessica, DC 22943"
"Kimberly Goodman","prangel@salazar.org","674-713-9658","9241 Jeremy Port, West Daniel, IL 53044"
"Olivia Hernandez","jennifermontoya@white.com","870-817-5607","503 Garcia Court, Heathland, NC 51839"
"Kathy Rodriguez","kevinharmon@krueger-burnett.com","151-227-7620","5970 Lewis Road, West Julie, NJ 44478"
"Michael Grant","kellylopez@yahoo.com","703-656-3080","Unit 9177 Box 6737, DPO AE 46188"
"Bradley Coleman","mpham@hotmail.com","545-607-8594","93418 Nicole Junction Apt. 984, Christineborough, UT 75492"


In [15]:
cur.execute('DROP TABLE IF EXISTS employees')

In [16]:
#if size[0] < 48000: # only execute if we are missing data
cur.execute('DROP TABLE IF EXISTS employees')
cur.execute('''CREATE TABLE employees (
            name varchar,
            email varchar, 
            phone_number varchar,
            address varchar,
            PRIMARY KEY (name, email));''')



cur.execute("""COPY employees(name,email,phone_number,address) 
                FROM '/encrypted_vol/jupyter_data/airflow/data/employees.csv'
                DELIMITER ',' CSV HEADER;""")

In [17]:
cur.execute('SELECT * FROM employees')

In [18]:
cur.fetchone()

('Chad Spears',
 'zacharyerickson@yahoo.com',
 '557-303-4631',
 '742 Jessica Cliffs, Port Stephenstad, TN 06314')

### Insert into email log table

In [19]:
!head -n 2 /encrypted_vol/jupyter_data/airflow/data/2019-08-23.csv

guid,datetime,sender,recipients,subject,attachments,rule,action,attach_cnt,rcpt_cnt,email_size,log_file,log_key_id
aRFWSXGwyJiTxPGMwzZSJcdRjWtpjCqB,2019-08-23 00:00:01+00:00,nsanchez@gilbert-wade.com,"['rodriguezsabrina@yahoo.com', 'ymcbride@hotmail.com', 'gberry@hotmail.com', 'karen09@hotmail.com']",Position or expect someone radio have.,['H:/him/technology/memory/many/sport/claim.inf'],AND,WHOLE_MAINTAIN_STOP,1,4,26100063,201901.gz,


In [20]:
cur.execute('DROP TABLE IF EXISTS email_logs')

In [21]:
cur.execute('''CREATE TABLE IF NOT EXISTS email_logs 
            (guid varchar, 
            "datetime" TIMESTAMP, 
            sender varchar, 
            recipients varchar, 
            subject varchar, 
            attachments varchar, 
            rule varchar, 
            action varchar, 
            attach_cnt int, 
            rcpt_cnt int, 
            email_size int, 
            log_file varchar, 
            log_key_id varchar,
            PRIMARY KEY (guid, sender));''')

In [22]:
cur.execute('''COPY email_logs(guid,datetime,sender,recipients,subject,attachments,rule,action,attach_cnt,rcpt_cnt,email_size,log_file,log_key_id)
                FROM '/encrypted_vol/jupyter_data/airflow/data/2019-08-22.csv'
                DELIMITER ',' CSV HEADER;''')


In [23]:
cur.execute('SELECT * FROM email_logs LIMIT 2')

In [24]:
cur.fetchall()

[('bBJKeUiocwuufpKQYWpWFIoABLbMmEfX',
  datetime.datetime(2019, 8, 22, 0, 0, 11),
  'dadams@hotmail.com',
  "['lcherry@hotmail.com']",
  'Will wait how ok whether why. Read avoid foreign if.',
  '[]',
  'CONGRESS',
  'STOP_TRADITIONAL',
  0,
  1,
  11473169,
  '201908.gz',
  None),
 ('yaWaJKfSYqIpZUZfditdxEBmvNuqkrac',
  datetime.datetime(2019, 8, 22, 0, 0, 12),
  'amy99@austin-alvarado.org',
  "['lsnyder@hotmail.com', 'joneschristopher@yahoo.com']",
  'Democrat risk bed support hear feel.',
  "['Z:/different/get/phone/a/risk.rdb']",
  'STATION',
  'COMMUNITY_TRAVEL',
  1,
  2,
  12063584,
  '201909.gz',
  None)]

In [25]:
get_table_col_names(conn, 'employees')

['name', 'email', 'phone_number', 'address']

### Insert into blended table

In [26]:
cur.execute('DROP TABLE IF EXISTS blended_email_logs')

In [27]:
cur.execute('''CREATE TABLE IF NOT EXISTS blended_email_logs (
                guid varchar, 
                "datetime" TIMESTAMP, 
                sender varchar, 
                recipients varchar, 
                subject varchar, 
                attachments varchar, 
                rule varchar, 
                action varchar, 
                attach_cnt int, 
                rcpt_cnt int, 
                email_size int, 
                log_file varchar, 
                log_key_id varchar, 
                name varchar,
                phone_number varchar,
                address varchar,
                PRIMARY KEY(guid, sender));''')

In [28]:
cur.execute('''insert into blended_email_logs(
                        guid,
                        datetime,
                        sender,
                        recipients,
                        subject,
                        attachments,
                        rule,
                        action,
                        attach_cnt,
                        rcpt_cnt,
                        email_size,
                        log_file,
                        log_key_id,
                        name,
                        phone_number,
                        address
)
SELECT 
  el.guid,el.datetime,el.sender,el.recipients,el.subject,el.attachments,el.rule,el.action,el.attach_cnt,el.rcpt_cnt,el.email_size,el.log_file,el.log_key_id,
  e.name,e.phone_number,e.address
FROM email_logs el
LEFT JOIN employees e ON el.sender = e.email''')


In [33]:
cur.execute('''SELECT count(*) FROM blended_email_logs WHERE datetime::date = date '2019-08-22' ''')

In [34]:
cur.fetchone()

(7480,)

In [35]:
cur.execute('''SELECT count(*) FROM blended_email_logs''')

In [36]:
cur.fetchone()

(7480,)

In [37]:
conn.close()